In [77]:
import imaplib
import base64
import os
import email
import xlrd
import xlwt
import openpyxl
from email.message import EmailMessage

In [78]:
class Email:
    def __init__(self, sender, subject, body):
        self.sender = sender
        self.subject = subject
        self.body = body
        
    def summary(self):
        print("Email From: " + self.sender)
        print("Email Subject: " + self.subject)
        print("Email Body: " + self.body)


In [79]:
#Set up for connection to outlook IMAP server
imap_clients = {
    "outlook" : ("imap-mail.outlook.com", 993)
}
loginFile = open("credentials.txt", "r")
connectionInformation = {}
for line in loginFile:
    key = line.split(":")[0]
    value = line.split(":")[1].rstrip("\n")
    connectionInformation[key] = value

client = connectionInformation["client"]
#Connect and login to outlook mailbox
mailbox = imaplib.IMAP4_SSL(imap_clients[client][0], imap_clients[client][1])
username = connectionInformation["username"]
password = connectionInformation["password"]
mailbox.login(username, password)

('OK', [b'LOGIN completed.'])

In [80]:
#Set the current mailbox to search from
mailbox.select('Inbox')
#Start going through unread emails 
unread_emails = []
type, data = mailbox.search(None, '(UNSEEN)')
mail_ids = data[0]
id_list = mail_ids.split()
temp = data[0].split()
if type == "OK":
    for num in temp:
        typ, data = mailbox.fetch(num, '(RFC822)' )
        raw_email = data[0][1]
        for response_part in data:
            if isinstance(response_part, tuple):
                #Create new Email object and add it to unread_emails array
                msg = email.message_from_string(response_part[1].decode('utf-8'))
                email_subject = msg['subject']
                email_from = msg['from']
                email_body = msg.get_payload()[0].get_payload()
                unread_email = EmailMessage()
                unread_email['Subject'] = email_subject
                unread_email['From'] = email_from
                unread_email.set_content(email_body)
                unread_emails.append(unread_email)

In [81]:
#for email in unread_emails:
#    print(email.summary())
#body = msg.get_payload()[0].get_payload()
#data_index = body.index("Name")
#body = body[data_index:].rsplit("\r\n")
#body

In [82]:
mailbox.close()
mailbox.logout()

('BYE', [b'Microsoft Exchange Server IMAP4 server signing off.'])

In [83]:
workbook = openpyxl.load_workbook("templates.xlsx")
customers = workbook["Customers"]

In [140]:
#Create param dict
for email in unread_emails:
    param_dict = {}
    email_body = email.get_content()#.splitlines()
    #email_body = email.body.replace("\r\n\r\n", "\n")
    #email_body = email.body.replace("\r\n", "\n")
    #email_body = email_body.replace("\n\n", " \n")
    #email_body = email.body.replace("\n", "")
    #email_body = email_body.rsplit(": ")
    #print(email_body)
    
    key= ""
    value = ""
    for character in email_body:
        if character != '\n':
            key = key + character
            
        if character == '\n':
            print("one line " + key)
            key = ""
            

one line First Name: Antony
one line Last Name: Darsuma
one line 
one line Source: codeninjas.com
one line Primary Email: icerain@uwalumni.com
one line Primary Phone: 3464123526
one line City, State: ,
one line 
one line Child First Name: Julius
one line Child Last Name: Darsuma
one line 
one line Comments: Additional Information Child: Julius Darsuma Referrer:
one line https://www.codeninjas.com/tx-cypress
one line First Name: Jes
one line Last Name: Byboth
one line 
one line Source: codeninjas.com
one line Primary Email: jesbyboth@gmail.com
one line Primary Phone: 9794504780
one line City, State: ,
one line 
one line Child First Name: Aiden
one line Child Last Name: Byboth
one line 
one line Comments: Additional Information Inquired About: Code Ninjas CREATE Program
one line Slug: code-ninjas-create Child: Aiden Byboth Referrer:
one line https://www.codeninjas.com/tx-cypress/code-ninjas-create/request-information


In [ ]:
    
    for param in email.body:
        #print(param)
        if ": " in param:
            (key, value) = param.split(": ")[0], param.split(": ")[1]
            param_dict[key] = value
    print(param_dict)

In [8]:
#Insert information into excel sheet 
key_dict = []
for row in customers.iter_cols(min_row=1, max_col=customers.max_column, max_row=2, values_only=True):
    key_dict.append(row[0])

new_customer_row = customers.max_row + 1
for current_col in range(customers.max_column):
    if key_dict[current_col] in param_dict:
        customers.cell(row=new_customer_row, column=current_col + 1).value = param_dict[key_dict[current_col]]
        

In [9]:
workbook.save("templates.xlsx")